### Generate Input

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os

# ==========================================
# 1. CONFIGURATION
# ==========================================
# Path sesuai struktur folder kamu
INPUT_CSV   = "../data/simulation/INPUT.csv"
REF_OUT_CSV = "../data/simulation/REF.csv"
WEIGHTS_DIR = "../data/model_csv/model_1"
SCALE       = 1024.0  # Q6.10 Fixed Point

# RTL Padding Config untuk DEC1 & DEC2
# Sesuai RTL generator_v3.v: PAD_TOP=2, PAD_BOTTOM=3, PAD_LEFT=2, PAD_RIGHT=3
PAD_CFG_RTL = (2, 3, 2, 3) # (Left, Right, Top, Bot) order untuk PyTorch F.pad

# ==========================================
# 2. HARDWARE EMULATION FUNCTIONS
# ==========================================
def to_q6_10_numpy(tensor):
    """Convert Float Tensor back to Q6.10 Integer"""
    arr = tensor.detach().cpu().numpy()
    return np.clip(np.round(arr * SCALE), -32768, 32767).astype(np.int32)

def rtl_leaky_relu(x):
    return F.leaky_relu(x, negative_slope=13.0/64.0)

def rtl_trans_conv_padded(x, w, b, pad_order):
    """
    Emulasi Decoder V3: Upsample -> Zero Pad -> Conv Valid
    """
    # 1. Upsample (Insert Zeros)
    B, C, H, W = x.shape
    up = torch.zeros((B, C, H*2, W*2), dtype=x.dtype, device=x.device)
    up[:, :, 0::2, 0::2] = x
    
    # 2. Padding Manual (Sesuai RTL zero_pad_layer)
    padded = F.pad(up, pad_order, mode='constant', value=0)
    
    # 3. Conv Valid
    return F.conv2d(padded, w, bias=b, padding=0)

# ==========================================
# 3. MODEL ARCHITECTURE (GENERATOR V3)
# ==========================================
class GeneratorV3_Ref(torch.nn.Module):
    def __init__(self, w_dir):
        super().__init__()
        self.weights = {}
        self.load_layer("enc1", 3, w_dir)
        self.load_layer("enc2", 4, w_dir)
        self.load_layer("enc3", 4, w_dir)
        self.load_layer("dec1", 4, w_dir)
        self.load_layer("dec2", 4, w_dir)
        self.load_layer("out",  3, w_dir)

    def load_layer(self, name, k_size, w_dir):
        # Load weights sesuai nama file di folder model_1
        w_path = os.path.join(w_dir, f"{name}_weight.csv")
        b_path = os.path.join(w_dir, f"{name}_bias.csv")
        
        if not os.path.exists(w_path):
            print(f"⚠️ Warning: {w_path} missing, using random.")
            w_np = np.random.randn(1, 1, k_size, k_size)
            b_np = np.array([0.0])
        else:
            w_flat = pd.read_csv(w_path, header=None).values.flatten()
            b_val  = pd.read_csv(b_path, header=None).values.flatten()[0]
            w_np = w_flat.reshape(1, 1, k_size, k_size)
            b_np = np.array([b_val])
            
        self.weights[f"{name}_w"] = torch.tensor(w_np, dtype=torch.float32)
        self.weights[f"{name}_b"] = torch.tensor(b_np, dtype=torch.float32)

    def forward(self, x):
        w = self.weights
        # Enc1: 32 -> 30
        x = F.conv2d(x, w["enc1_w"], bias=w["enc1_b"]); x = rtl_leaky_relu(x)
        # Enc2: 30 -> 14 (Stride 2)
        x = F.conv2d(x, w["enc2_w"], bias=w["enc2_b"], stride=2); x = rtl_leaky_relu(x)
        # Enc3: 14 -> 6 (Stride 2)
        x = F.conv2d(x, w["enc3_w"], bias=w["enc3_b"], stride=2); x = rtl_leaky_relu(x)
        
        # Dec1: 6 -> 14 (Upsample + Pad + Conv)
        x = rtl_trans_conv_padded(x, w["dec1_w"], w["dec1_b"], PAD_CFG_RTL); x = rtl_leaky_relu(x)
        # Dec2: 14 -> 30 (Upsample + Pad + Conv)
        x = rtl_trans_conv_padded(x, w["dec2_w"], w["dec2_b"], PAD_CFG_RTL); x = rtl_leaky_relu(x)
        
        # Out: 30 -> 28
        x = F.conv2d(x, w["out_w"], bias=w["out_b"])
        x = torch.tanh(x)
        return x

# ==========================================
# 4. EXECUTION
# ==========================================
if not os.path.exists(INPUT_CSV):
    print(f"❌ Error: Input file {INPUT_CSV} not found!")
else:
    print(f"🔄 Loading input: {INPUT_CSV}")
    raw_input = np.loadtxt(INPUT_CSV, dtype=np.float32)
    
    # Reshape Flat CSV -> (N, 1, 32, 32)
    num_pixels = 32 * 32
    num_frames = len(raw_input) // num_pixels
    input_reshaped = raw_input[:num_frames*num_pixels].reshape(num_frames, 1, 32, 32)
    
    # De-quantize (Q6.10 Integer -> Float)
    input_tensor = torch.tensor(input_reshaped / SCALE)
    
    # Run Inference
    model = GeneratorV3_Ref(WEIGHTS_DIR)
    model.eval()
    
    print("🧠 Running PyTorch Model...")
    with torch.no_grad():
        out_tensor = model(input_tensor)
        
    # Quantize Result (Float -> Q6.10 Integer)
    out_fixed = to_q6_10_numpy(out_tensor)
    
    # Save
    np.savetxt(REF_OUT_CSV, out_fixed.flatten(), fmt='%d')
    print(f"✅ Reference saved to: {REF_OUT_CSV}")
    print(f"   Processed {num_frames} frames.")